# Router Chain Example

This notebook demonstrates how to use LangChain's Router Chain to route queries to different specialized prompts based on the input type. The example implements a system that can:

1. **Analyze Requirements**: Routes to a requirement analyst prompt that converts high-level product requirements into GHERKIN format acceptance criteria
2. **Create Test Plans**: Routes to a tester prompt that develops comprehensive test strategies and plans
3. **Generate Test Automation**: Routes to an automation expert prompt that converts manual test cases into automated tests using Playwright and Robot Framework

## Key Components

- Multiple specialized prompts for different roles
- Router chain to determine which prompt to use
- Example inputs showing the full workflow from requirements to test automation

## Prerequisites

- OpenAI API key
- LangChain library
- Python environment with JupyterLab/Notebook



In [58]:
prompt_requirement_analyst = """You are a requirement analyst who is great in understanding the high level product requirements, finding gaps or contradictory requirements, asking questions to fill in missing gaps, and finally documenting them in acceptance criteria in GHERKIN format in an organized way. You have over 20 years of experience in data processing / data analyst domain.

Here is the high level product feature requirement:
{input}
"""

prompt_tester = """You are an expert tester who is great in discovering edge cases and bugs. You come up with scenarios that are closer to customer usecases since you have over 20 years of experience in data processing (etl) domain. You would be provided with the findings of the business analyst and your main goal is to use product's acceptance criteria and come up with concrete test strategy and test plan for the given project. Do not just limit to the acceptance criteria but also consider other findings by the business analyst such as open questions, gaps etc. Keep track of any pending items in the test plan so that it is under radar. Remember, you are not just translating the acceptance crietria into test cases but also adding more value by coming up with more test cases that are not covered in the acceptance criteria since you are an expert tester.

Here is the output of the business analyst:
{input}
"""

prompt_automation = """You are an expert automation tester who is great in writing automated tests. You expertise lies in Playwright for e2e automated tests and Robot for api tests. Your main goal is to convert the manual tests into automated tests that are reliable and easy to maintain following test best practices. The tester would provide you with the complete test plan and strategy and you need to automate the documented tests in it. Automate the tests according to the test pyramid shifting left to api as much as possible with minimal e2e tests. Generate automated tests in different sections. 

Here is the output of the tester:
{input}
"""

prompts = [
    {
        "name": "Requirement Analyst",
        "description": "Expert in translating high level product requirements into acceptance criteria in Gherkin format",
        "prompt_template": prompt_requirement_analyst,
    },
    {
        "name": "Tester",
        "description": "Expert in writing test strategy and test plan using the acceptance criteria given for the given project",
        "prompt_template": prompt_tester,
    },
    {
        "name": "Automation",
        "description": "Expert in writing automated tests using playwright and ROBOT",
        "prompt_template": prompt_automation,
    }
]

In [59]:
import subprocess

OPEN_AI_KEY = subprocess.run('bash -l -c "getpassw $OPEN_AI_API_KEY"', shell=True, capture_output=True, text=True)
OPEN_AI_KEY = OPEN_AI_KEY.stdout.strip()
LLM_MODEL = "gpt-4o-mini-2024-07-18"

print("")

In [60]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, api_key=OPEN_AI_KEY, model=LLM_MODEL)


In [61]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt_chains = {}
for p in prompts:
    prompt_template = ChatPromptTemplate.from_template(template=p["prompt_template"])
    llmChain = LLMChain(llm=llm, prompt=prompt_template)
    prompt_chains[p["name"]] = llmChain

prompt_name_and_description = "\n".join([f"{p['name']}: {p['description']}" for p in prompts])


In [62]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain
from langchain.chains.router import MultiPromptChain

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language \
model, the model should be able to route to the correct prompt based \
on the input. You will be provided with the name of the prompts and \
high level purpose of the prompt in its description.\

Prompts:
{prompt_name_and_description}

Response Format/Output:
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string // The name of the prompt that user provided OR DEFAULT if no prompt is found
    "next_inputs": string // The input that user provided
}}}}
```

Input:
{{input}}
"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(prompt_name_and_description=prompt_name_and_description)
router_prompt = PromptTemplate(template=router_template, 
                                 input_variables=["input"],
                                 output_parser=RouterOutputParser()
                                )
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)
chain = MultiPromptChain(router_chain = router_chain, 
                         destination_chains=prompt_chains,
                         default_chain=default_chain, 
                         verbose=True)

In [63]:
# Store chain outputs
chain_outputs = {}

# First chain run - Get requirements analysis
chain_outputs['requirements'] = chain.run("Feature: A ETL tool to additionally support filtering of rows using new advance filter expression")
print("Requirements Analysis Output:")
print(chain_outputs['requirements'])


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Requirement Analyst: {'input': 'Feature: A ETL tool to additionally support filtering of rows using new advance filter expression'}
> Finished chain.
Requirements Analysis Output:
To effectively analyze the high-level product requirement for the ETL tool that supports filtering of rows using a new advanced filter expression, we need to break down the requirement, identify potential gaps or contradictions, and formulate acceptance criteria in GHERKIN format.

### High-Level Requirement Breakdown
1. **Feature Overview**: The ETL tool should allow users to filter rows based on advanced filter expressions.
2. **User Needs**: Users need the ability to define complex filtering criteria to refine the data being processed.
3. **Functionality**: The filtering mechanism should support various logical and comparison operators, as well as functions for more advanced filtering.

### Questions to Fill in Gaps
1. **What types of data sources will the ETL tool support for filtering?**
2. **What specif

In [65]:
# Second chain run - Get test plan using requirements output
chain_outputs['test_plan'] = chain.run(f"""
Generate the test plan for the requirements gather by the business analyst:
{chain_outputs['requirements']}
""")
print("\nTest Plan Output:")
print(chain_outputs['test_plan'])


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Tester: {'input': 'Generate the test plan for the requirements gather by the business analyst: To effectively analyze the high-level product requirement for the ETL tool that supports filtering of rows using a new advanced filter expression, we need to break down the requirement, identify potential gaps or contradictions, and formulate acceptance criteria in GHERKIN format.\n\n### High-Level Requirement Breakdown\n1. **Feature Overview**: The ETL tool should allow users to filter rows based on advanced filter expressions.\n2. **User Needs**: Users need the ability to define complex filtering criteria to refine the data being processed.\n3. **Functionality**: The filtering mechanism should support various logical and comparison operators, as well as functions for more advanced filtering.\n\n### Questions to Fill in Gaps\n1. **What types of data sources will the ETL tool support for filtering?**\n2. **What specific advanced filter expressions are expected? (e.g., AND, OR, NOT, etc.)**\n3

In [66]:
# Third chain run - Get test automation using test plan output
chain_outputs['automation'] = chain.run(f"""
Automated the tests provided in this test plan:
{chain_outputs['test_plan']}
""")
print("\nTest Automation Output:")
print(chain_outputs['automation'])


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Automation: {'input': 'Automated the tests provided in this test plan:\n### Test Plan for Advanced Row Filtering in ETL Tool\n\n#### 1. **Test Plan Overview**\nThis test plan outlines the strategy and approach for testing the advanced filtering feature in the ETL tool. The goal is to ensure that the filtering functionality meets the acceptance criteria and addresses the user needs effectively while identifying potential edge cases and gaps.\n\n#### 2. **Objectives**\n- Validate the functionality of advanced filter expressions.\n- Ensure the user interface is intuitive and meets user expectations.\n- Assess performance under various data loads.\n- Identify and handle error scenarios gracefully.\n- Confirm that saved filter expressions can be reused effectively.\n\n#### 3. **Scope of Testing**\n- **In-Scope**:\n  - Functionality of filter expressions (simple and complex).\n  - Logical and comparison operators.\n  - User interface for filter definition.\n  - Error handling for invalid exp